Resolve the filtered phrases back to the original mid section (with expressive timing and dynamics).

In [1]:
import os

import numpy as np

import pretty_midi

In [2]:
filtered_dir = './samples_out'
dataset_dir = './POP909-Dataset-master'
out_dir = './resolved_909'

In [4]:
def reorder_notes(notes):
    # reorder note by starting time
    return sorted(notes, key=lambda x: x.start)

def notes_to_pit_bucket(notes):
    # convert pretty_midi notes to 128*1 np arrays
    out = np.zeros((128, 1))
    for note in notes:
        out[note.pitch] += 1
    return out

for i, seg_name in enumerate(os.listdir(filtered_dir)):
    song_id = seg_name[:-2]
    quantized_path = f'{filtered_dir}/{seg_name}/accompaniment_track.mid'
    quantized_notes = reorder_notes(pretty_midi.PrettyMIDI(quantized_path).instruments[0].notes)
    
    original_path =f'{dataset_dir}/POP909/{song_id}/{song_id}.mid'
    original_mid = pretty_midi.PrettyMIDI(original_path)
    original_notes = []
    for inst in original_mid.instruments:
        if inst.name == 'PIANO' or inst.name == 'BRIDGE':
            original_notes += inst.notes
        original_notes = reorder_notes(original_notes)
    original_bpms = original_mid.get_tempo_changes()[1]

    # To find the original midi sequence:
    # Match the pitch of the first note (try all of them if there are more than one)
    # Compare the bucket of pitches
    quantized_bucket = notes_to_pit_bucket(quantized_notes)
    
    # Find the first pitch(es)
    first_pits = []
    for note in quantized_notes:
        if note.start == 0:
            first_pits.append(note.pitch)

    best_candidate, best_score = 0, 999
    for idx, note in enumerate(original_notes):
        if note.pitch in first_pits:
            candidate = original_notes[idx: idx + len(quantized_notes)]
            original_bucket = notes_to_pit_bucket(candidate)
            
            # Simple L1 distance suffices here
            score = np.sum(np.abs(original_bucket - quantized_bucket))
            if score < best_score:
                best_score = score
                best_candidate = candidate

    if best_score > 2:
        print(seg_name, best_score)

    # Adjust for the starting offset and tempo
    offset = best_candidate[0].start
    for note in best_candidate:
        note.start -= offset
        note.end -= offset
    
    # Find the best bpm by end time
    best_bpm = 0
    best_score = 999
    for bpm in original_bpms:
        bpm = float(bpm)
        if bpm > 0:
            end_t = best_candidate[-1].start * bpm / 80
            score = abs(end_t - quantized_notes[-1].start)
            if score < best_score:
                best_score = score
                best_bpm = bpm

    for note in best_candidate:
        note.start *= best_bpm / 80
        note.end *= best_bpm / 80

    if best_score > 1:
        print(seg_name, best_score)

    # Write the mid
    inst = pretty_midi.Instrument(0)
    inst.notes = best_candidate
    out_mid = pretty_midi.PrettyMIDI(initial_tempo=80)
    out_mid.instruments.append(inst)
    out_mid.write(f'{out_dir}/{seg_name}.mid')

    

132_A 4.0
473_X 4.0
476_X 8.0
476_X 1.2375000000000007
888_A 4.0
888_X 4.0
